In [1]:
! nvidia-smi

Wed Jun  1 07:41:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install pytorch-tabnet optuna 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 53.5 MB/s 
     |████████████████████████████████| 209 kB 67.7 MB/s 
     |████████████████████████████████| 81 kB 11.8 MB/s 
     |████████████████████████████████| 78 kB 8.4 MB/s 
     |████████████████████████████████| 112 kB 60.8 MB/s 
     |████████████████████████████████| 146 kB 71.7 MB/s 
     |████████████████████████████████| 49 kB 7.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=9239689881f9346f520965c2edd3b2e7a7d08ead8003b1fb00965a6788e5a0d1
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [3]:
import pandas as pd
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score,classification_report
import optuna as opt
import torch
import os
import joblib

In [4]:
def make_save_cv_model(i,model_name,model,best_params,optim,output_path="./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/cross_validated_models"):

    ''' This function saves cross validation model in the corresponding directory ( if the path does not exist it creates the path for it'''


    if os.path.exists(os.path.join(output_path,f"{i}_{model_name}_{optim}")):
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/model_params.txt"),"w+") as file:
            file.write(str(best_params))
    else:
        os.mkdir(os.path.join(output_path,f"{i}_{model_name}_{optim}"))
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/model_params.txt"),"w+") as file:
            file.write(str(best_params))

In [5]:
def train(fold_dict,fold,model_name,sc_df,tar_col,optim,optim_trial,k_folds=10,tar_cols="",verbose=1):

    ''' this function is used to train the model with parameters optimization using optuna and cross validation using stratified k_folds'''

    y = sc_df[tar_col]
    x = sc_df.drop([tar_col],axis=1)
    model_name = model_name 
    def objective(trial):
      train_index = fold_dict[fold]["train"]
      test_index = fold_dict[fold]["test"]
      clf = TabNetClassifier(n_d=trial.suggest_int("n_d", 8, 64),
                              n_a =trial.suggest_int("n_a", 8, 64),
                              n_steps = trial.suggest_int("n_steps",3,10),
                              gamma =trial.suggest_float("gamma", 1.0, 2.0),
                              n_independent = trial.suggest_int("n_independent",1,5),
                              n_shared = trial.suggest_int("n_shared",1,5),
                              momentum = trial.suggest_float("momentum", 0.01, 0.4),
                              optimizer_fn = torch.optim.Adam,
                              # scheduler_fn = torch.optim.lr_scheduler,
                              # scheduler_params = {"gamma" :trial.suggest_float("sch-gamma", 0.5, 0.95), "step_size": trial.suggest_int("sch_step_size", 10, 20, 2)},
                              verbose = verbose,
                              device_name = "auto"
                              )
      # print(f" train_index :: {train_index}")
      # print(f" test_index :: {test_index}")
      X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
      # print(X_train.shape, X_test.shape)
      X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
      Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
      Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
      print(Y_train.shape, Y_test.shape)
      clf.fit(X_train, Y_train,
              eval_set=[(X_test, Y_test)],
              eval_metric=['accuracy'])
      Y_pred = clf.predict(X_test)
      print(classification_report(Y_test, Y_pred, labels=[x for x in range(6)]))
      acc = accuracy_score(Y_pred, Y_test)
      return acc

    print(f"Starting optimization for fold : [{fold}/{k_folds}]")
    study = opt.create_study(direction='maximize')
    study.optimize(objective, n_trials=optim_trial)
    best_params = study.best_params
    print(f" Best params for fold : [{fold}/{k_folds}]")
    print(best_params)
    joblib.dump(best_params,f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/best_params/comp/fold_{fold}_best_params.z")
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/best_params/fold_{fold}_best_params.txt", "w+") as file:file.write(str(best_params))
    print(f"Saved best_params at : outputs/{model_name}/best_params/fold_{fold}_best_params.txt")
    train_index = fold_dict[fold]["train"]
    test_index = fold_dict[fold]["test"]
    X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
    # print(X_train.shape, X_test.shape)
    X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
    Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
    Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
    clf_model = TabNetClassifier(**study.best_params)
    clf_model.fit(X_train,Y_train)
    Y_pred = clf_model.predict(X_test)
    clf_report = classification_report(Y_test, Y_pred, labels=[x for x in range(6)])
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/classification_report/{model_name}_{fold}_classification_report.txt","w+") as file:file.write(str(clf_report))
    accuracy = accuracy_score(Y_pred, Y_test)
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/{model_name}_{fold}_accuracy_score.txt","w+") as file:file.write(f" accuracy :: {str(accuracy)}")
    try:
        print("[++] Saving the model and parameters in corresponding directories")
        make_save_cv_model(fold,model_name,clf_model,best_params,optim=optim)
    except:
        print("[-] Failed to save the model")

In [6]:
use_df = pd.read_csv("./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/data/trainable_scaled_balanced.csv")
tar_col = "PCE_categorical"
model_name = "pytorch_tabnet"
optimizer = "Adam"
fold_dict = joblib.load("./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/inputs/fold_vals/fold_data.z")
fold = 1

In [7]:
train(fold_dict = fold_dict,
      fold = fold,
      model_name=model_name,
      sc_df=use_df,
      tar_col=tar_col,
      optim=optimizer,
      optim_trial = 15)
print(f"[++] Ended the training process for fold {fold}")

[I 2022-06-01 07:41:22,945] A new study created in memory with name: no-name-5c45f491-6d83-48f4-99b8-9d8f97ba46ab


Starting optimization for fold : [1/10]
Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 1.22036 | val_0_accuracy: 0.60125 |  0:00:02s
epoch 1  | loss: 0.88052 | val_0_accuracy: 0.60471 |  0:00:04s
epoch 2  | loss: 0.84606 | val_0_accuracy: 0.63968 |  0:00:06s
epoch 3  | loss: 0.78967 | val_0_accuracy: 0.66019 |  0:00:08s
epoch 4  | loss: 0.74945 | val_0_accuracy: 0.66321 |  0:00:10s
epoch 5  | loss: 0.74237 | val_0_accuracy: 0.67703 |  0:00:12s
epoch 6  | loss: 0.73309 | val_0_accuracy: 0.6766  |  0:00:14s
epoch 7  | loss: 0.72491 | val_0_accuracy: 0.66991 |  0:00:16s
epoch 8  | loss: 0.71649 | val_0_accuracy: 0.67098 |  0:00:18s
epoch 9  | loss: 0.69516 | val_0_accuracy: 0.69041 |  0:00:22s
epoch 10 | loss: 0.6845  | val_0_accuracy: 0.70315 |  0:00:26s
epoch 11 | loss: 0.67932 | val_0_accuracy: 0.70099 |  0:00:28s
epoch 12 | loss: 0.6752  | val_0_accuracy: 0.69646 |  0:00:30s
epoch 13 | loss: 0.66516 | val_0_accuracy: 0.70682 |  0:00:32s
epoch 14 | loss: 0.6571  | val_0_accuracy:

[I 2022-06-01 07:44:35,674] Trial 0 finished with value: 0.790587219343696 and parameters: {'n_d': 47, 'n_a': 39, 'n_steps': 5, 'gamma': 1.2195459451766517, 'n_independent': 1, 'n_shared': 2, 'momentum': 0.015289150352253908}. Best is trial 0 with value: 0.790587219343696.


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       825
           1       0.65      0.60      0.63       826
           2       0.82      0.92      0.87       822
           3       1.00      1.00      1.00       779
           4       0.54      0.54      0.54       761
           5       0.67      0.64      0.66       619

    accuracy                           0.79      4632
   macro avg       0.78      0.78      0.78      4632
weighted avg       0.79      0.79      0.79      4632

Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 1.7586  | val_0_accuracy: 0.46438 |  0:00:04s
epoch 1  | loss: 1.1732  | val_0_accuracy: 0.54879 |  0:00:09s
epoch 2  | loss: 1.11059 | val_0_accuracy: 0.52051 |  0:00:14s
epoch 3  | loss: 1.03727 | val_0_accuracy: 0.58614 |  0:00:19s
epoch 4  | loss: 0.88532 | val_0_accuracy: 0.61161 |  0:00:24s
epoch 5  | loss: 0.83696 | val_0_accuracy: 0.644   |  0:00:29s
epoch 6  | loss: 0.7946  | val_0_accuracy: 

[I 2022-06-01 07:52:44,211] Trial 1 finished with value: 0.7933937823834197 and parameters: {'n_d': 8, 'n_a': 52, 'n_steps': 7, 'gamma': 1.4712770362880545, 'n_independent': 4, 'n_shared': 3, 'momentum': 0.26200944992984126}. Best is trial 1 with value: 0.7933937823834197.


...

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       825
           1       0.56      0.53      0.54       826
           2       0.82      0.84      0.83       822
           3       1.00      1.00      1.00       779
           4       0.47      0.58      0.52       761
           5       0.65      0.49      0.56       619

    accuracy                           0.75      4632
   macro avg       0.75      0.74      0.74      4632
weighted avg       0.75      0.75      0.75      4632

Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 1.36335 | val_0_accuracy: 0.59909 |  0:00:02s
epoch 1  | loss: 0.93431 | val_0_accuracy: 0.60622 |  0:00:04s
epoch 2  | loss: 0.85742 | val_0_accuracy: 0.63731 |  0:00:07s
epoch 3  | loss: 0.84112 | val_0_accuracy: 0.63212 |  0:00:09s
epoch 4  | loss: 0.82779 | val_0_accuracy: 0.61874 |  0:00:11s
epoch 5  | loss: 0.81333 | val_0_accuracy: 0.65458 |  0:00:14s
epoch 6  | loss: 0.77883 | val_0_accuracy: 

[I 2022-06-01 08:50:49,924] Trial 13 finished with value: 0.7705094991364422 and parameters: {'n_d': 35, 'n_a': 29, 'n_steps': 5, 'gamma': 1.2472486283743442, 'n_independent': 2, 'n_shared': 2, 'momentum': 0.3304379753219197}. Best is trial 3 with value: 0.8037564766839378.


              precision    recall  f1-score   support

           0       0.99      1.00      1.00       825
           1       0.59      0.57      0.58       826
           2       0.81      0.91      0.86       822
           3       1.00      1.00      1.00       779
           4       0.50      0.49      0.50       761
           5       0.67      0.60      0.63       619

    accuracy                           0.77      4632
   macro avg       0.76      0.76      0.76      4632
weighted avg       0.77      0.77      0.77      4632

Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 1.28427 | val_0_accuracy: 0.57535 |  0:00:02s
epoch 1  | loss: 0.93214 | val_0_accuracy: 0.60881 |  0:00:04s
epoch 2  | loss: 0.84403 | val_0_accuracy: 0.62435 |  0:00:07s
epoch 3  | loss: 0.8033  | val_0_accuracy: 0.63644 |  0:00:09s
epoch 4  | loss: 0.76119 | val_0_accuracy: 0.67206 |  0:00:12s
epoch 5  | loss: 0.73806 | val_0_accuracy: 0.67422 |  0:00:14s
epoch 6  | loss: 0.7199  | val_0_accuracy: 

[I 2022-06-01 08:53:18,852] Trial 14 finished with value: 0.7692141623488774 and parameters: {'n_d': 44, 'n_a': 29, 'n_steps': 3, 'gamma': 1.6184874724169172, 'n_independent': 3, 'n_shared': 4, 'momentum': 0.32071202667105614}. Best is trial 3 with value: 0.8037564766839378.


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       825
           1       0.57      0.55      0.56       826
           2       0.82      0.88      0.85       822
           3       1.00      1.00      1.00       779
           4       0.51      0.55      0.53       761
           5       0.69      0.58      0.63       619

    accuracy                           0.77      4632
   macro avg       0.76      0.76      0.76      4632
weighted avg       0.77      0.77      0.77      4632

 Best params for fold : [1/10]
{'n_d': 17, 'n_a': 55, 'n_steps': 3, 'gamma': 1.2906043466163537, 'n_independent': 2, 'n_shared': 1, 'momentum': 0.3578040087905696}
Saved best_params at : outputs/pytorch_tabnet/best_params/fold_1_best_params.txt
Device used : cuda
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 1.093   |  0:00:01s
epoch 1  | loss: 0.8258  |  0:00:02s
epoch 2  | loss: 0.76969 |  0:00:03s
epoch 3  

Fold 0 has started running on 20-05-22 


Fold 0 has completed sucessfully on 17:00 20-05-22

Fold 1 has started running at 15:15 21-05-22

Fold 2 has started running at 09:45 22-05-22

Fold 2 has completed sucessfully on 10:58 22-05-22

Fold 3 has started running at 18:40 22-05-22

Fold 3 has completed sucessfully on 22-05-22

Fold 4 completed sucessfully on 21:04 on 22-05-22

Fold 5 started at 18:21 on 23-05-22

Fold 5 completed sucessfully on 19:44 on 23-05-22

Fold 6 started at 12:53 on 24-05-22

Fold 6 has completed at 14:14 on 24-05-22

Fold 7 started at 14:18 on 24-05-22

Fold 7 execution failed due to colab gpu time limit

Fold 7 trial 1 started at 11:00 on 25-05-22

Fold 7 has completed sucessfully at 12:14 on 25-05-22 

Fold 8 has started at 9:38 on 26-05-22

Fold 8 filed due to interrupted internet connection

Fold 8 trial 1 started at 13:38 on 26-05-22

Fold 8 has successfully executed at 15:33 on 26-05-22

Fold 9 has started at 13:35 on 27-05-22

Fold 9 has completed at 14:55 on 27-05-22

Editing with rectified dataset witout duplicacy because of space values

Fold 0 started at 13:21 on 28-05-22

Fold 0 completed sucessfully at 14:46 on 28-05-22

Fold 1 Failed to run due to some index error

_____ Restarting the training process again due to data distribution failure____



Fold 0 started at 10:47 on 30-05-22

Fold 0 completed successfully at 12:30 on 30-05-22

Fold 1 started at 8:27 on 31-05-22

Fold 1 execution failed due to runtime disconnection

Fold 1 started again at 9:38 on 31-05-22

Fold 1 execution failed due to gpu disconnect 

Fold 1 started again at 8:36 on 1-06-22

Fold 1 execution failed due to network disconnection

Fold 1 started again at 13:11 on 01-06-22

Fold 1 has succesfully executed at 14:25 on 01-06-22